In [7]:
import os
import torchvision.models as models

# Clear the model cache
cache_dir = os.path.expanduser('~/.cache/torch/hub/checkpoints')
if os.path.exists(cache_dir):
    for file in os.listdir(cache_dir):
        os.remove(os.path.join(cache_dir, file))

# Now reload the models
resnet50 = models.resnet50(pretrained=True)
resnet101 = models.resnet101(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/aryan/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 27.6MB/s]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /Users/aryan/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:06<00:00, 27.8MB/s] 


In [11]:
import torch
import torchvision.models as models
import os

def get_condition_number(tensor):
    if tensor.dim() > 2:
        tensor = tensor.view(tensor.size(0), -1)  
    u, s, vh = torch.linalg.svd(tensor, full_matrices=False)
    condition_number = s[0] / s[-1]
    return condition_number.item()

def compute_max_condition_number(model):
    max_condition_number = 0
    max_layer = ""
    for name, param in model.named_parameters():
        if 'weight' in name and param.dim() > 1:  
            condition_number = get_condition_number(param.data)
            if condition_number > max_condition_number:
                max_condition_number = condition_number
                max_layer = name
    return max_layer, max_condition_number

def load_model_again(model_fn):
    for _ in range(5):  
        try:
            model = model_fn(pretrained=True)
            return model
        except Exception as e:
            print(f"Error loading model: {e}. Retrying...")
    raise RuntimeError("Failed to load the model after multiple attempts.")

cache_dir = os.path.expanduser('~/.cache/torch/hub/checkpoints')
if os.path.exists(cache_dir):
    for file in os.listdir(cache_dir):
        os.remove(os.path.join(cache_dir, file))

resnet50 = load_model_again(models.resnet50)
resnet101 = load_model_again(models.resnet101)

max_layer_resnet50, max_condition_number_resnet50 = compute_max_condition_number(resnet50)
max_layer_resnet101, max_condition_number_resnet101 = compute_max_condition_number(resnet101)

print("Maximum Condition Number for ResNet50:")
print(f"Layer: {max_layer_resnet50}, Condition Number: {max_condition_number_resnet50}")
print("\nMax Condition Number for ResNet101:")
print(f"Layer: {max_layer_resnet101}, Condition Number: {max_condition_number_resnet101}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/aryan/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 27.7MB/s]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torchvision/models/_util

Maximum Condition Number for ResNet50:
Layer: layer1.0.conv1.weight, Condition Number: 27317304.0

Max Condition Number for ResNet101:
Layer: layer1.0.conv1.weight, Condition Number: 50743968.0


In [3]:
import torch
import torchvision.models as models
import os
from tqdm import tqdm

def get_condition_number(tensor):
    if tensor.dim() > 2:
        tensor = tensor.view(tensor.size(0), -1)  
    u, s, vh = torch.linalg.svd(tensor, full_matrices=False)
    condition_number = s[0] / s[-1]
    return condition_number.item()

def compute_max_condition_number(model):
    max_condition_number = 0
    for name, param in model.named_parameters():
        if 'weight' in name and param.dim() > 1:  
            condition_number = get_condition_number(param.data)
            if condition_number > max_condition_number:
                max_condition_number = condition_number
    return max_condition_number

def load_model_again(model_fn):
    for _ in range(3):  # Try three times
        try:
            model = model_fn(pretrained=True)
            return model
        except Exception as e:
            print(f"Error loading model: {e}. Retrying...")
    raise RuntimeError("Failed to load the model after multiple attempts.")

cache_dir = os.path.expanduser('~/.cache/torch/hub/checkpoints')
if os.path.exists(cache_dir):
    for file in os.listdir(cache_dir):
        os.remove(os.path.join(cache_dir, file))

resnet50 = load_model_again(models.resnet50)
resnet101 = load_model_again(models.resnet101)

max_condition_number_resnet50 = compute_max_condition_number(resnet50)
max_condition_number_resnet101 = compute_max_condition_number(resnet101)

print(f"Max Condition Number for ResNet50: {max_condition_number_resnet50}")
print(f"Max Condition Number for ResNet101: {max_condition_number_resnet101}")

if max_condition_number_resnet50 < max_condition_number_resnet101:
    print("ResNet50 has the lowest maximum condition number.")
else:
    print("ResNet101 has the lowest maximum condition number.")

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/aryan/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:03<00:00, 29.2MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /Users/aryan/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:06<00:00, 29.4MB/s]


Max Condition Number for ResNet50: 27317304.0
Max Condition Number for ResNet101: 50743968.0
ResNet50 has the lowest maximum condition number.


In [4]:
import torch
import torchvision.models as models

import os

def get_condition_number(tensor):
    if tensor.dim() > 2:
        tensor = tensor.view(tensor.size(0), -1)  # Reshape to 2D tensor
    u, s, vh = torch.linalg.svd(tensor, full_matrices=False)
    condition_number = s[0] / s[-1]
    return condition_number.item()

def compute_layerwise_condition_numbers(model):
    condition_numbers = {}
    for name, param in model.named_parameters():
        if 'weight' in name and param.dim() > 1:  
            condition_numbers[name] = get_condition_number(param.data)
    return condition_numbers

def load_model_with_retry(model_fn):
    for _ in range(3):  # Try three times
        try:
            model = model_fn(pretrained=True)
            return model
        except Exception as e:
            print(f"Error loading model: {e}. Retrying...")
    raise RuntimeError("Failed to load the model after multiple attempts.")

# Clear the model cache
cache_dir = os.path.expanduser('~/.cache/torch/hub/checkpoints')
if os.path.exists(cache_dir):
    for file in os.listdir(cache_dir):
        os.remove(os.path.join(cache_dir, file))

# Load ResNet models with retries
resnet50 = load_model_with_retry(models.resnet50)
resnet101 = load_model_with_retry(models.resnet101)

# Compute condition numbers for ResNet50
condition_numbers_resnet50 = compute_layerwise_condition_numbers(resnet50)
# Compute condition numbers for ResNet101
condition_numbers_resnet101 = compute_layerwise_condition_numbers(resnet101)

# Print the results
print("Condition Numbers for ResNet50:")
for layer, condition_number in condition_numbers_resnet50.items():
    print(f"{layer}: {condition_number}")

print("\nCondition Numbers for ResNet101:")
for layer, condition_number in condition_numbers_resnet101.items():
    print(f"{layer}: {condition_number}")


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/aryan/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:04<00:00, 23.3MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /Users/aryan/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:07<00:00, 23.5MB/s] 


Condition Numbers for ResNet50:
conv1.weight: 15679958.0
layer1.0.conv1.weight: 27317304.0
layer1.0.conv2.weight: 10742037.0
layer1.0.conv3.weight: 20600190.0
layer1.0.downsample.0.weight: 7935351.0
layer1.1.conv1.weight: 8357168.0
layer1.1.conv2.weight: 7.552454948425293
layer1.1.conv3.weight: 10.541668891906738
layer1.2.conv1.weight: 11.183545112609863
layer1.2.conv2.weight: 6.060380935668945
layer1.2.conv3.weight: 10.609183311462402
layer2.0.conv1.weight: 28.318416595458984
layer2.0.conv2.weight: 6.450717926025391
layer2.0.conv3.weight: 10.562396049499512
layer2.0.downsample.0.weight: 78.14446258544922
layer2.1.conv1.weight: 89.0654067993164
layer2.1.conv2.weight: 40.38343811035156
layer2.1.conv3.weight: 26.33506202697754
layer2.2.conv1.weight: 13.442540168762207
layer2.2.conv2.weight: 7.293711185455322
layer2.2.conv3.weight: 8.750475883483887
layer2.3.conv1.weight: 10.036197662353516
layer2.3.conv2.weight: 5.163841247558594
layer2.3.conv3.weight: 8.585745811462402
layer3.0.conv1.we